In [1]:

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import pymc as pm

In [4]:
pxi_df = pd.read_csv(r"data/pxi_results.csv")

pxi_df

,participant_id,game_name,trial,mode,difficulty,pxi_AA,pxi_CH,pxi_EC,pxi_GR,pxi_PF,pxi_AUT,pxi_CUR,pxi_IMM,pxi_MAS,pxi_MEA,pxi_ENJ,STIMES,TIMES
0,P001,WordZapper,1.0,2.0,0.0,Slightly agree,Agree,Slightly agree,Disagree,Disagree,Agree,Agree,Strongly Agree,Disagree,Neutral,Agree,1.744629e+12,1.744629e+12
1,P001,Turmoil,1.0,3.0,0.0,Agree,Agree,Agree,Slightly agree,slightly disagree,Agree,Agree,Strongly Agree,Agree,Neutral,Agree,1.744629e+12,1.744629e+12
2,P001,Boxing,1.0,0.0,2.0,Slightly agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Slightly agree,Neutral,Strongly Agree,Agree,Neutral,Agree,1.744629e+12,1.744629e+12
3,P001,WordZapper,2.0,1.0,3.0,Slightly agree,Agree,Slightly agree,Disagree,slightly disagree,slightly disagree,Slightly agree,Strongly Agree,Disagree,Neutral,Neutral,1.744629e+12,1.744630e+12
4,P001,Turmoil,2.0,0.0,0.0,Slightly agree,Agree,Agree,Slightly agree,Slightly agree,Agree,Agree,Strongly Agree,Agree,Neutral,Slightly agree,1.744630e+12,1.744630e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,P011,Turmoil,2.0,2.0,0.0,Slightly agree,Agree,Agree,Strongly Agree,Strongly Agree,slightly disagree,Agree,Strongly Agree,Agree,Agree,Strongly Agree,1.746111e+12,1.746111e+12
90,P011,WordZapper,2.0,0.0,3.0,Slightly agree,Disagree,Strongly disagree,Strongly disagree,slightly disagree,slightly disagree,slightly disagree,Agree,Disagree,Slightly agree,slightly disagree,1.746111e+12,1.746111e+12
91,P011,Turmoil,3.0,0.0,0.0,Slightly agree,slightly disagree,Strongly Agree,Strongly Agree,Agree,Slightly agree,Agree,Strongly Agree,Slightly agree,Agree,Agree,1.746111e+12,1.746111e+12
92,P011,WordZapper,3.0,2.0,1.0,slightly disagree,Disagree,Disagree,Disagree,Disagree,Disagree,slightly disagree,Strongly Agree,Strongly disagree,Agree,slightly disagree,1.746111e+12,1.746111e+12


# Boxing

In [8]:
pxi_df_box = pxi_df[pxi_df["game_name"] == "Boxing"].drop(columns="game_name")
pxi_df_box.head()

,participant_id,trial,mode,difficulty,pxi_AA,pxi_CH,pxi_EC,pxi_GR,pxi_PF,pxi_AUT,pxi_CUR,pxi_IMM,pxi_MAS,pxi_MEA,pxi_ENJ,STIMES,TIMES
2,P001,1.0,0.0,2.0,Slightly agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Slightly agree,Neutral,Strongly Agree,Agree,Neutral,Agree,1.744629e+12,1.744629e+12
5,P001,2.0,0.0,3.0,Agree,Agree,Strongly Agree,Agree,Agree,Agree,Agree,Strongly Agree,Agree,Neutral,Agree,1.744630e+12,1.744630e+12
6,P002,1.0,0.0,2.0,Agree,Strongly disagree,Strongly Agree,Strongly Agree,Strongly Agree,Slightly agree,Slightly agree,Strongly Agree,Strongly Agree,Neutral,Agree,1.745489e+12,1.745490e+12
9,P002,2.0,0.0,3.0,Agree,Strongly disagree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Agree,Slightly agree,Agree,1.745490e+12,1.745490e+12
14,P002,3.0,0.0,1.0,Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Agree,Agree,Strongly Agree,1.745491e+12,1.745491e+12
